# 구글 드라이브 연동

In [ ]:
from google.colab import drive
drive._mount('/content/drive')

# 라이브러리

In [54]:
import numpy as np
import pandas as pd
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

# Google sheet 연동을 위한 인증

In [22]:
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

# 데이터 로드

In [27]:
# 구글 시트 파일명으로 불러오기
file_name = 'PJ Coding Test Sample'
data = gc.open(file_name)
# 첫번째 sheet 불러오기
sheet1 = data.sheet1
sheet1

<Worksheet '시트1' id:867315742>

In [59]:
# 구글 sheet의 데이터를 데이터 프레임으로 변환
df = pd.DataFrame(sheet1.get_all_records())

# Category1 마지막 행 데이터 '개이정보' 오타 수정
df['Category1'][-1:] = '개인정보'

# 데이터 확인
df

,Question,Category1,Category2,Category3
0,사용자 가입은 어디서 하나요?,일반문의,가입/탈퇴,가입
1,탈퇴는 어디서 하나요?,일반문의,가입/탈퇴,탈퇴
2,사용자 암호 기준을 알려주세요,일반문의,가입/탈퇴,비밀번호
3,비밀번호를 잊어버렸어요. 어디서 바꾸나요?,일반문의,가입/탈퇴,비밀번호
4,키오스크앱이 설치가 안되요,키오스크앱,설치,
5,그룹수업을 개설했다가 일괄취소를 하려고 하는데 에러가 나요,사용법,그룹수업,일괄취소
6,그룹수업의 강사가 탈퇴했는데 어디서 바꾸나요?,사용법,그룹수업,강사관리
7,사용자 가입은 어디서 하나요?,일반문의,가입/탈퇴,가입
8,탈퇴는 어디서 하나요?,일반문의,가입/탈퇴,탈퇴
9,사용자 암호 기준을 알려주세요,일반문의,가입/탈퇴,비밀번호


# 1. 1. Problem 1

## (1) Unique Category List 만들기

In [91]:
category_list = []
for val in df.values :
    # 비어있는 데이터 확인
    empty = np.where(val == '')[0]
    # 비어있는 category가 있을 경우
    if empty.size > 0 :
        category_list.append(" > ".join(val[1:empty[0]]))
    # 비어있는 category가 없을 경우
    else:
        category_list.append(" > ".join(val[1:]))

# 중복값 제거(Unique)
unique_list = list(set(category_list))

# 내림차순 정렬
unique_list.sort()

# Unique Category List 확인
unique_list

['개인정보 > 동의서  > 제3자 제공동의',
 '개인정보 > 동의서 > 수집목적',
 '개인정보 > 동의서 > 제공항목',
 '사용법 > 그룹수업 > 강사관리',
 '사용법 > 그룹수업 > 일괄취소',
 '일반문의 > 가입/탈퇴 > 가입',
 '일반문의 > 가입/탈퇴 > 비밀번호',
 '일반문의 > 가입/탈퇴 > 탈퇴',
 '키오스크앱 > 설치']

## (2) category list의 index와 질문의 pair 속성으로 CSV 생성

In [ ]:
# 각 질문의 카테고리와 일치하는 값의 index를 위 df에 추가하기
category_idx = []
for val in df.values :
    empty = np.where(val == '')[0]
    if empty.size > 0 :
        category = " > ".join(val[1:empty[0]])
        # 각 질문에 해당하는 category와 일치하는 값 index 추출
        idx = np.where(np.array(unique_list) == category)[0][0]
        category_idx.append(idx)
    else:
        category = " > ".join(val[1:])
        # 각 질문에 해당하는 category와 일치하는 값 index 추출
        idx = np.where(np.array(unique_list) == category)[0][0]
        category_idx.append(idx)

# Index 컬럼 추가
df['Index'] = category_idx

# 질문과 index pair csv로 저장
df.to_csv('./category_index_info.csv', columns=['Question', 'Index'], index=False, encoding='utf-8-sig')